# Análisis de Sentimiento en Textos
Alan Badillo Salas (badillo.soft@hotmail.com)

En estre proyecto vamos a utilizar las redes neuronales de tipo RNN (Redes Reuronales Recurrentes), para poder predecir si un texto (una reseña en inglés) será positiva o negativa. Para ello necesitamos una base de entrenamiento la cuál fue recreada del artículo: https://medium.com/@dclengacher/keras-lstm-recurrent-neural-networks-c1f5febde03d.

Podemos descargar el corpus directo en: https://github.com/zaidalyafeai/Browser-Sentiment-Classification.

El punto de partida en nuestro análisis es un corpus que contiene `7086` reseñas obtenidas de IMDB y consisten en un breve texto en inglés y una etiqueta sobre si la reseña es positiva (`1`) o si es negativa (`0`).

Vamos a cargar el corpus en pandas para analizarlo.

In [62]:
import pandas as pd

corpus = pd.read_csv("http://badillosoft.com/corpus.csv")

corpus = corpus.filter(items=["text", "label"])

print(corpus.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7086 entries, 0 to 7085
Data columns (total 2 columns):
text     7086 non-null object
label    7086 non-null int64
dtypes: int64(1), object(1)
memory usage: 110.8+ KB
None


In [63]:
print(corpus.head())

                                                text  label
0            The Da Vinci Code book is just awesome.      1
1  this was the first clive cussler i've ever rea...      1
2                   i liked the Da Vinci Code a lot.      1
3                   i liked the Da Vinci Code a lot.      1
4  I liked the Da Vinci Code but it ultimatly did...      1


Para no perder generalidad vamos a revolver aleatoriamente el corpus.

In [64]:
corpus = corpus.sample(frac=1)

print(corpus.head())

                                                   text  label
3479           Brokeback Mountain was an AWESOME movie.      1
5535  I hate Harry Potter, that daniel wotshisface n...      0
3539           Brokeback Mountain was an AWESOME movie.      1
1920  I like Mission Impossible movies because you n...      1
3153  Derek and I saw 3 movies, Brokeback Mountain, ...      1


El aprendizaje se va a dar a tráves de secuencias de palabras, es decir, de alguna forma tenemos que representar una secuencia de palabras por un vector de tamaño fijo.

Ejemplo: Si tenemos el texto `"i liked the Da Vinci Code a lot"`, lo primero que tenemos que hacer es representar ese texto por un vector de palabras: `["i", "liked", "the", "da", "vinci", "code", "a", "lot"]`. Observa que el vector vector de palabras está en minúsculas y no contiene caracteres especiales, sólo letras. Aún con esto la red neuronal no es capaz de procesar dichas palabras, por lo que tenemos que generar un diccionario de palabras y sustituir la palabra por su índice en el diccionario, entonces, si el diccionario es como `{"a": 20, "code": 100, ...}`, podemos generar el vector de palabras como: `[56, 28, 33, 17, 109, 0, 1502]`. Es decir, que cada palabra va a ser sustituida por su índice en el diccionario, el diccionario va a contener los índices de cada palabra diferente.

Vamos a definir una función llamada `text_to_vec` que reciba un texto (una cadena de caracteres) y nos devuelva un vector de palabras.

In [65]:
import re

def text_to_vec(text):
    return re.sub(r"[^a-z\s]", "", text.lower()).split()

text_to_vec("I liked the Da Vinci Code a lot")

['i', 'liked', 'the', 'da', 'vinci', 'code', 'a', 'lot']

Vamos a formar una lista que contenga los textos representados como vectores de palabras.

In [66]:
texts = corpus["text"].map(text_to_vec)

print(texts)

3479       [brokeback, mountain, was, an, awesome, movie]
5535    [i, hate, harry, potter, that, daniel, wotshis...
3539       [brokeback, mountain, was, an, awesome, movie]
1920    [i, like, mission, impossible, movies, because...
3153    [derek, and, i, saw, movies, brokeback, mounta...
5386    [this, quiz, sucks, and, harry, potter, sucks,...
2402    [i, am, going, to, start, reading, the, harry,...
4209    [da, vinci, code, up, up, down, down, left, ri...
3109                      [i, loved, brokeback, mountain]
6978    [she, helped, me, bobbypin, my, insanely, cool...
6491                   [brokeback, mountain, was, boring]
6438    [she, helped, me, bobbypin, my, insanely, cool...
5303    [harry, potter, dragged, draco, malfoy, s, tro...
835                       [i, love, the, da, vinci, code]
2658    [harry, potter, is, awesome, i, dont, care, if...
5728    [is, it, just, me, or, does, harry, potter, suck]
6182    [then, dinner, with, min, and, rosie, and, bro...
6657    [then,

Ahora necesitamos una lista que contenga todas las palabras de nuestros textos.

In [67]:
words = []

for text_vec in texts:
    words.extend(text_vec)
    
print("Se encontraron {} palabras".format(len(words)))
print("Primeras 100 palabras: {}".format(words[:100]))

Se encontraron 75401 palabras
Primeras 100 palabras: ['brokeback', 'mountain', 'was', 'an', 'awesome', 'movie', 'i', 'hate', 'harry', 'potter', 'that', 'daniel', 'wotshisface', 'needs', 'a', 'fucking', 'slap', 'brokeback', 'mountain', 'was', 'an', 'awesome', 'movie', 'i', 'like', 'mission', 'impossible', 'movies', 'because', 'you', 'never', 'know', 'whos', 'on', 'the', 'right', 'side', 'derek', 'and', 'i', 'saw', 'movies', 'brokeback', 'mountain', 'which', 'was', 'beautiful', 'i', 'almost', 'cried', 'this', 'quiz', 'sucks', 'and', 'harry', 'potter', 'sucks', 'ok', 'bye', 'i', 'am', 'going', 'to', 'start', 'reading', 'the', 'harry', 'potter', 'series', 'again', 'because', 'that', 'is', 'one', 'awesome', 'story', 'da', 'vinci', 'code', 'up', 'up', 'down', 'down', 'left', 'right', 'left', 'right', 'b', 'a', 'suck', 'i', 'loved', 'brokeback', 'mountain', 'she', 'helped', 'me', 'bobbypin', 'my', 'insanely']


Ahora tenemos una lista con todas las palabras en todos nuestros textos, por lo que formaremos un conjunto menor con las palabras direntes (las palabras únicas).

In [68]:
unique_words = list(set(words))

print("Se encontraron {} palabras diferentes".format(len(unique_words)))
print("Primeras 100: {}".format(unique_words[:100]))

Se encontraron 2193 palabras diferentes
Primeras 100: ['suspenseful', 'four', 'sucksi', 'asian', 'mcgarther', 'hating', 'genre', 'captain', 'hate', 'weekda', 'stinks', 'snowing', 'spec', 'created', 'walks', 'voted', 'scorebrokeback', 'trousers', 'lore', 'lord', 'sorry', 'worth', 'codeother', 'updated', 'oceans', 'betterwe', 'every', 'jack', 'bringing', 'tickets', 'school', 'prize', 'clothed', 'undercover', 'enjoy', 'jamaica', 'tired', 'warns', 'hanging', 'feathers', 'messy', 'second', 'street', 'friendships', 'n', 'even', 'hide', 'christ', 'musiclove', 'new', 'increasing', 'ever', 'disney', 'told', 'deemed', 'kicked', 'hero', 'zach', 'gayer', 'never', 'here', 'nanny', 'dork', 'kudos', 'study', 'changed', 'controversy', 'credit', 'dudeee', 'aka', 'postponed', 'changes', 'campaign', 'blashpemies', 'julia', 'lynne', 'bro', 'total', 'plot', 'spoke', 'would', 'geisha', 'music', 'preview', 'type', 'until', 'holy', 'oscar', 'haha', 'yahoo', 'award', 'disruption', 'adult', 'excellent', 'hold',

Ahora vamos a definir dos funciones útiles, una llamada `word_frequency` que nos devolverá la frecuencia de una palabra y otra llamada `word_index` que nos devolverá el índice de la palabra en el diccionario de palabras únicas.

In [69]:
def word_frequency(word):
    return words.count(word)

word_frequency("the")

3221

In [84]:
def word_index(word):
    if not word in unique_words:
        return 0
    return unique_words.index(word) + 1

word_index("the")

104

Opcionalmente podemos filtrar las palabras únicas que tengan un frecuancia mayor a un umbral.

In [81]:
THRESH = 5

unique_words = list(filter(lambda word: word_frequency(word) >= THRESH, unique_words))

print("Se encontraron {} palabras diferentes".format(len(unique_words)))

Se encontraron 450 palabras diferentes


Ahora que conocemos la frecuencia de una palabra en el corpus y su posición en el diccionario de palabras diferentes, procederemos a generar secuencias de palabras. Podemos definir el tamaño de la cuencia manualmente o utilizar el tamaño de secuencia de palabras más grande para nuestros textos. El tamaño de secuencia es el número de palabras en el texto.

Obtenemos el número de palabras por texto.

In [85]:
texts_len = list(map(len, texts))
# texts_len = list(map(lambda text_vec: len(text_vec), texts))

print("Tamaños de las secuencias para los primeros 100 textos: {}".format(texts_len[:100]))

Tamaños de las secuencias para los primeros 100 textos: [6, 11, 6, 14, 13, 9, 17, 14, 4, 21, 4, 21, 32, 6, 11, 9, 14, 15, 12, 29, 5, 12, 8, 7, 15, 10, 9, 6, 34, 5, 6, 28, 4, 6, 8, 5, 21, 11, 5, 6, 30, 14, 7, 25, 10, 5, 7, 15, 15, 4, 21, 12, 11, 28, 14, 14, 4, 4, 7, 7, 6, 4, 7, 4, 22, 4, 6, 8, 12, 4, 6, 4, 12, 9, 12, 4, 8, 4, 6, 16, 21, 7, 7, 11, 5, 12, 4, 5, 13, 10, 21, 4, 25, 7, 5, 6, 6, 6, 8, 11]


Con la lista anterior podemos calcular cuál es la secuencia máxima de palabras, es decir, cuál es el texto que tiene más palabras.

In [86]:
max_seq = max(texts_len) # podemos sino utilizar un número fijo, ej. 100

print("La máxima secuencia de palabras es: {}".format(max_seq))

La máxima secuencia de palabras es: 38


Con anterior podemos definir una función llamada `text_to_seq` que convierta un texto (en su forma de vector de palabras) en una secuencia de índices respecto al diccionario de palabras únicas.

Ejemplo: Tenemos el texto `["i", "love", "brokeback", "mountain"]` representado como un vector de palabras, la secuencia generada debería quedar como `[0, 0, 0, 0, 0, 0, 0, 0, ..., 0, 2053,  632,  775, 1394]`. Observa que la secuencia contine los índices de cada palabra y rellena de ceros las demás posiciones para dejar una secuencia de tamaño fijo igual a `max_seq`, poniendo las palabras al final de la secuencia.

In [87]:
import numpy as np

def text_to_seq(text_vec):
    seq = [0] * max_seq
    n = len(text_vec)
    seq[-n:] = list(map(word_index, text_vec))
    return np.array(seq)
    
text_to_seq(["i", "love", "brokeback", "mountain"])

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0, 427, 144, 170, 329])

Ahora ya podemos convertir todos textos a secuencias, para formar la matriz de entrenamiento `x_train`.

In [95]:
texts_seq = list(map(text_to_seq, texts))

print("La primer secuencia quedaría como: {}".format(texts_seq[0]))

La primer secuencia quedaría como: [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0 170 329 127 418
 150 314]


A partir de las secuancias, podemos calcular `x_train` y `x_test` para realizar el aprendizaje.

In [96]:
n = len(texts_seq)
k = int(0.95 * n)

x_train = np.array(texts_seq[:k])
x_test = np.array(texts_seq[k:])

print("Se analizarán {}/{} secuencias".format(len(x_train), n))
print("Se validarán {}/{} secuencias".format(len(x_test), n))

Se analizarán 6731/7086 secuencias
Se validarán 355/7086 secuencias


In [97]:
print("Primeras 3 secuencias para el entrenamiento")
print(x_train[:3])

Primeras 3 secuencias para el entrenamiento
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0 170 329 127 418
  150 314]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0 427   1 259 173 181  50 345 274 276
  338 186]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0 170 329 127 418
  150 314]]


Para formar `y_train` y `y_test` realizamos un proceso similar, utilizando las etiquetas del corpus.

In [98]:
y_train = corpus["label"][:k]
y_test = corpus["label"][k:]

print("Primeras 3 etiquetas para el entrenamiento:")
print(y_train[:3])

Primeras 3 etiquetas para el entrenamiento:
3479    1
5535    0
3539    1
Name: label, dtype: int64


Ahora ya tenemos todos los elementos para entrenar nuestra red RNN-GRU (`x_train`, `x_test`, `y_train`, `y_test`).

Vamos a crear una red neuronal recurrente RNN con unidad de memoria GRU.

In [103]:
from keras.models import Sequential
from keras.layers import Embedding, GRU, Dense

model = Sequential()

# Defininimos la capa de transformación de índices a vectores
EMBEDDING_SIZE = 8
model.add(Embedding(
    input_dim=len(unique_words) + 1,
    output_dim=EMBEDDING_SIZE,
    input_length=max_seq
))

# Definimos las capas GRU
model.add(GRU(16, return_sequences=True))
model.add(GRU(8, return_sequences=True))
model.add(GRU(4))

# Definimos la capa Densa para el entrenamiento
model.add(Dense(1, activation="sigmoid"))

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 38, 8)             3608      
_________________________________________________________________
gru_10 (GRU)                 (None, 38, 16)            1200      
_________________________________________________________________
gru_11 (GRU)                 (None, 38, 8)             600       
_________________________________________________________________
gru_12 (GRU)                 (None, 4)                 156       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 5         
Total params: 5,569
Trainable params: 5,569
Non-trainable params: 0
_________________________________________________________________
None


Entonces procedemos a entrenar el modelo.

In [104]:
model.fit(x_train, y_train.values, epochs=10, batch_size=32)

Epoch 1/10
6731/6731 [==============================] - 24s 4ms/step - loss: 0.3866 - acc: 0.8214
Epoch 2/10
6731/6731 [==============================] - 20s 3ms/step - loss: 0.0902 - acc: 0.9816
Epoch 3/10
6731/6731 [==============================] - 21s 3ms/step - loss: 0.0467 - acc: 0.9902
Epoch 4/10
6731/6731 [==============================] - 21s 3ms/step - loss: 0.0296 - acc: 0.9941
Epoch 5/10
6731/6731 [==============================] - 21s 3ms/step - loss: 0.0229 - acc: 0.9957
Epoch 6/10
6731/6731 [==============================] - 21s 3ms/step - loss: 0.0182 - acc: 0.9970
Epoch 7/10
6731/6731 [==============================] - 21s 3ms/step - loss: 0.0178 - acc: 0.9958
Epoch 8/10
6731/6731 [==============================] - 21s 3ms/step - loss: 0.0119 - acc: 0.9976
Epoch 9/10
6731/6731 [==============================] - 21s 3ms/step - loss: 0.0103 - acc: 0.9984
Epoch 10/10
6731/6731 [==============================] - 20s 3ms/step - loss: 0.0088 - acc: 0.9987


In [105]:
metrics = model.evaluate(x_test, y_test)

print(metrics)

355/355 [==============================] - 1s 3ms/step
[0.04572702128584431, 0.9859154929577465]


Ahora con la red RNN entrenada podemos pensar en un texto (en inglés) para predecir si va a ser una reseña positiva o negativa.

In [133]:
def sentiment(text):
    text_vec = text_to_vec(text)
    text_seq = text_to_seq(text_vec)
    predict = model.predict(text_seq.reshape(1, -1))
    return predict[0][0]

def sentiment_positive(text):
    return sentiment(text) >= 0.5

def sentiment_negative(text):
    return sentiment(text) < 0.5

def sentiment_label(text):
    return "POSITIVE" if sentiment_positive(text) else "NEGATIVE"

In [134]:
text = "I hate you Harry Potter"

print(sentiment_label(text))

NEGATIVE


In [135]:
text = "I love you Harry Potter"

print(sentiment_label(text))

POSITIVE


In [136]:
text = "Impossible Mission"

print(sentiment_label(text))

NEGATIVE
